In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ✅ set your repo root once
REPO_ROOT = Path("/Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection").resolve()

DATA_DIR = REPO_ROOT / "data/processed"
ART_DIR  = REPO_ROOT / "artifacts"
FIG_DIR  = REPO_ROOT / "docs/figures/models/06_model_comparison"
FIG_DIR.mkdir(parents=True, exist_ok=True)

VAL_PATH  = DATA_DIR / "val.csv"
TEST_PATH = DATA_DIR / "test.csv"

print("VAL exists:", VAL_PATH.exists(), VAL_PATH)
print("TEST exists:", TEST_PATH.exists(), TEST_PATH)
print("FIG_DIR:", FIG_DIR)


VAL exists: True /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/data/processed/val.csv
TEST exists: True /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/data/processed/test.csv
FIG_DIR: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison


In [2]:
def load_split(path: Path, target_col: str = "Class"):
    df = pd.read_csv(path)
    y = df[target_col].astype(int).values
    X = df.drop(columns=[target_col])
    return df, X, y

val_df, X_val_df, y_val = load_split(VAL_PATH)
test_df, X_test_df, y_test = load_split(TEST_PATH)

print("Val:", X_val_df.shape, "fraud:", int(y_val.sum()))
print("Test:", X_test_df.shape, "fraud:", int(y_test.sum()))


Val: (42721, 102) fraud: 56
Test: (42722, 102) fraud: 52


In [3]:
import joblib

XGB_MODEL_PATH = ART_DIR / "xgb_model.pkl"
XGB_METRICS_PATH = ART_DIR / "xgb_metrics.json"

xgb = joblib.load(XGB_MODEL_PATH)

with open(XGB_METRICS_PATH, "r", encoding="utf-8") as f:
    xgb_metrics = json.load(f)

xgb_thr = float(xgb_metrics["threshold"])
print("XGB threshold:", xgb_thr)


XGB threshold: 0.162


In [4]:
from sklearn.metrics import (
    roc_curve, precision_recall_curve, roc_auc_score, average_precision_score,
    confusion_matrix, precision_score, recall_score, f1_score
)

def eval_at_threshold(y_true, scores, thr):
    preds = (scores >= thr).astype(int)
    cm = confusion_matrix(y_true, preds)
    return {
        "cm": cm,
        "precision": precision_score(y_true, preds, zero_division=0),
        "recall": recall_score(y_true, preds, zero_division=0),
        "f1": f1_score(y_true, preds, zero_division=0),
        "auc": roc_auc_score(y_true, scores),
        "ap": average_precision_score(y_true, scores),
    }

xgb_val_scores  = xgb.predict_proba(X_val_df.values)[:, 1]
xgb_test_scores = xgb.predict_proba(X_test_df.values)[:, 1]

xgb_val_eval  = eval_at_threshold(y_val, xgb_val_scores, xgb_thr)
xgb_test_eval = eval_at_threshold(y_test, xgb_test_scores, xgb_thr)

xgb_val_eval, xgb_test_eval


({'cm': array([[42654,    11],
         [    6,    50]]),
  'precision': 0.819672131147541,
  'recall': 0.8928571428571429,
  'f1': 0.8547008547008547,
  'auc': 0.9980608896552878,
  'ap': 0.9083880297183039},
 {'cm': array([[42656,    14],
         [   10,    42]]),
  'precision': 0.75,
  'recall': 0.8076923076923077,
  'f1': 0.7777777777777778,
  'auc': 0.9926583259721296,
  'ap': 0.8514309352448828})

In [5]:
import os

AE_SUMMARY_PATH = REPO_ROOT / "docs/figures/models/05_autoencoder/ae_eval_summary.json"
with open(AE_SUMMARY_PATH, "r", encoding="utf-8") as f:
    ae_summary = json.load(f)

ae_thr = float(ae_summary["threshold"])
print("AE threshold:", ae_thr)
print("AE threshold source:", ae_summary.get("threshold_source"))

# ---- Try to load precomputed errors first ----
CANDIDATES = [
    ART_DIR / "ae_val_errors.npy",
    ART_DIR / "ae_test_errors.npy",
    REPO_ROOT / "docs/figures/models/05_autoencoder/ae_val_errors.npy",
    REPO_ROOT / "docs/figures/models/05_autoencoder/ae_test_errors.npy",
]

val_err_path = next((p for p in CANDIDATES if p.name == "ae_val_errors.npy" and p.exists()), None)
test_err_path = next((p for p in CANDIDATES if p.name == "ae_test_errors.npy" and p.exists()), None)

ae_val_scores = None
ae_test_scores = None

if val_err_path and test_err_path:
    ae_val_scores = np.load(val_err_path)
    ae_test_scores = np.load(test_err_path)
    print("✅ Loaded AE errors from:", val_err_path, test_err_path)
else:
    print("⚠️ Precomputed AE error arrays not found. Will try computing from model file...")


AE threshold: 2.413630972099965
AE threshold source: ae_threshold.txt
⚠️ Precomputed AE error arrays not found. Will try computing from model file...


In [6]:
if ae_val_scores is None or ae_test_scores is None:
    import tensorflow as tf

    # Try keras paths
    AE_MODEL_PATHS = [
        ART_DIR / "autoencoder_model.keras",
        ART_DIR / "autoencoder_model.h5",
    ]
    ae_model_path = next((p for p in AE_MODEL_PATHS if p.exists()), None)
    if ae_model_path is None:
        raise FileNotFoundError("No AE model file found (autoencoder_model.keras or .h5). Either save ae_*_errors.npy or keep the model artifact.")

    ae = tf.keras.models.load_model(ae_model_path)
    print("Loaded AE model:", ae_model_path)

    # Reconstruction error = mean squared error per row
    def recon_error(model, X):
        X_np = X.values.astype(np.float32)
        X_hat = model.predict(X_np, verbose=0)
        return np.mean((X_np - X_hat) ** 2, axis=1)

    ae_val_scores  = recon_error(ae, X_val_df)
    ae_test_scores = recon_error(ae, X_test_df)

    # Save for next time
    np.save(ART_DIR / "ae_val_errors.npy", ae_val_scores)
    np.save(ART_DIR / "ae_test_errors.npy", ae_test_scores)
    print("✅ Saved AE errors to artifacts/: ae_val_errors.npy, ae_test_errors.npy")


2025-12-24 22:37:37.643217: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-12-24 22:37:37.643254: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-12-24 22:37:37.643259: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-12-24 22:37:37.643276: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-24 22:37:37.643288: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Loaded AE model: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/artifacts/autoencoder_model.keras


2025-12-24 22:37:37.992922: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


✅ Saved AE errors to artifacts/: ae_val_errors.npy, ae_test_errors.npy


In [7]:
ae_val_eval  = eval_at_threshold(y_val, ae_val_scores, ae_thr)
ae_test_eval = eval_at_threshold(y_test, ae_test_scores, ae_thr)

ae_val_eval, ae_test_eval


({'cm': array([[42150,   515],
         [   32,    24]]),
  'precision': 0.04452690166975881,
  'recall': 0.42857142857142855,
  'f1': 0.08067226890756303,
  'auc': 0.9453797023321223,
  'ap': 0.02893819614950638},
 {'cm': array([[42362,   308],
         [   29,    23]]),
  'precision': 0.06948640483383686,
  'recall': 0.4423076923076923,
  'f1': 0.12010443864229765,
  'auc': 0.9271448144075283,
  'ap': 0.044535622758506796})

In [8]:
def plot_roc(y_true, score_a, score_b, label_a, label_b, title, out_path: Path):
    fpr_a, tpr_a, _ = roc_curve(y_true, score_a)
    fpr_b, tpr_b, _ = roc_curve(y_true, score_b)

    auc_a = roc_auc_score(y_true, score_a)
    auc_b = roc_auc_score(y_true, score_b)

    plt.figure()
    plt.plot(fpr_a, tpr_a, label=f"{label_a} (AUC={auc_a:.3f})")
    plt.plot(fpr_b, tpr_b, label=f"{label_b} (AUC={auc_b:.3f})")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

plot_roc(y_val,  xgb_val_scores,  ae_val_scores,  "XGBoost", "Autoencoder", "ROC (Validation)", FIG_DIR / "roc_val.png")
plot_roc(y_test, xgb_test_scores, ae_test_scores, "XGBoost", "Autoencoder", "ROC (Test)",       FIG_DIR / "roc_test.png")

print("Saved:", FIG_DIR / "roc_val.png")
print("Saved:", FIG_DIR / "roc_test.png")


Saved: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison/roc_val.png
Saved: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison/roc_test.png


In [9]:
def plot_pr(y_true, score_a, score_b, label_a, label_b, title, out_path: Path):
    p_a, r_a, _ = precision_recall_curve(y_true, score_a)
    p_b, r_b, _ = precision_recall_curve(y_true, score_b)

    ap_a = average_precision_score(y_true, score_a)
    ap_b = average_precision_score(y_true, score_b)

    plt.figure()
    plt.plot(r_a, p_a, label=f"{label_a} (AP={ap_a:.3f})")
    plt.plot(r_b, p_b, label=f"{label_b} (AP={ap_b:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

plot_pr(y_val,  xgb_val_scores,  ae_val_scores,  "XGBoost", "Autoencoder", "PR (Validation)", FIG_DIR / "pr_val.png")
plot_pr(y_test, xgb_test_scores, ae_test_scores, "XGBoost", "Autoencoder", "PR (Test)",       FIG_DIR / "pr_test.png")

print("Saved:", FIG_DIR / "pr_val.png")
print("Saved:", FIG_DIR / "pr_test.png")


Saved: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison/pr_val.png
Saved: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison/pr_test.png


In [10]:
def save_cm(cm, title, out_path: Path):
    plt.figure()
    plt.imshow(cm)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    for (i, j), v in np.ndenumerate(cm):
        plt.text(j, i, str(v), ha="center", va="center")
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()

save_cm(xgb_val_eval["cm"],  f"XGB CM (Val) @thr={xgb_thr:.3f}",  FIG_DIR / "cm_xgb_val.png")
save_cm(xgb_test_eval["cm"], f"XGB CM (Test) @thr={xgb_thr:.3f}", FIG_DIR / "cm_xgb_test.png")

save_cm(ae_val_eval["cm"],   f"AE CM (Val) @thr={ae_thr:.3f}",    FIG_DIR / "cm_ae_val.png")
save_cm(ae_test_eval["cm"],  f"AE CM (Test) @thr={ae_thr:.3f}",   FIG_DIR / "cm_ae_test.png")

print("Saved CM images into:", FIG_DIR)


Saved CM images into: /Users/lavanyasrinivas/Documents/AI-First-Preauth-Fraud-Detection/AI-First-Preauth-Fraud-Detection/docs/figures/models/06_model_comparison


In [11]:
summary = {
    "xgb": {
        "threshold": xgb_thr,
        "val":  {k: (v.tolist() if hasattr(v, "tolist") else v) for k, v in xgb_val_eval.items()},
        "test": {k: (v.tolist() if hasattr(v, "tolist") else v) for k, v in xgb_test_eval.items()},
    },
    "ae": {
        "threshold": ae_thr,
        "val":  {k: (v.tolist() if hasattr(v, "tolist") else v) for k, v in ae_val_eval.items()},
        "test": {k: (v.tolist() if hasattr(v, "tolist") else v) for k, v in ae_test_eval.items()},
    },
    "figures_saved_to": str(FIG_DIR),
}

summary_path = FIG_DIR / "model_comparison_summary.json"
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2)

pd.DataFrame({
    "Model": ["XGB (val)", "XGB (test)", "AE (val)", "AE (test)"],
    "Precision": [xgb_val_eval["precision"], xgb_test_eval["precision"], ae_val_eval["precision"], ae_test_eval["precision"]],
    "Recall":    [xgb_val_eval["recall"],    xgb_test_eval["recall"],    ae_val_eval["recall"],    ae_test_eval["recall"]],
    "F1":        [xgb_val_eval["f1"],        xgb_test_eval["f1"],        ae_val_eval["f1"],        ae_test_eval["f1"]],
    "AUC":       [xgb_val_eval["auc"],       xgb_test_eval["auc"],       ae_val_eval["auc"],       ae_test_eval["auc"]],
    "AP":        [xgb_val_eval["ap"],        xgb_test_eval["ap"],        ae_val_eval["ap"],        ae_test_eval["ap"]],
})


,Model,Precision,Recall,F1,AUC,AP
0,XGB (val),0.819672,0.892857,0.854701,0.998061,0.908388
1,XGB (test),0.750000,0.807692,0.777778,0.992658,0.851431
2,AE (val),0.044527,0.428571,0.080672,0.945380,0.028938
3,AE (test),0.069486,0.442308,0.120104,0.927145,0.044536
